In [0]:
from pyspark.sql import functions as F

s = spark.table("workspace.portfolio_jobs.silver_linkedin_jobs")

normalized = (
    s
    .withColumn(
        "posted_days_ago",
        F.when(F.col("job_posted_time").rlike(r"\\d+ day"), 
               F.regexp_extract("job_posted_time", r"(\\d+)", 1).cast("int"))
         .when(F.col("job_posted_time").rlike(r"\\d+ week"), 
               F.regexp_extract("job_posted_time", r"(\\d+)", 1).cast("int") * 7)
         .otherwise(F.lit(0))
    )
    .withColumn(
        "posted_date",
        F.date_sub(F.current_date(), F.col("posted_days_ago"))
    )
)

In [0]:
gold_trend = (
    normalized
    .groupBy("posted_date")
    .agg(F.count("*").alias("job_posts"))
    .orderBy("posted_date")
)

(gold_trend.write
 .mode("overwrite")
 .format("delta")
 .saveAsTable("workspace.portfolio_jobs.gold_post_trend"))


In [0]:
gold_companies = (s
  .groupBy("company_norm")
  .agg(F.count("*").alias("job_posts"))
  .orderBy(F.desc("job_posts"))
)

(gold_companies.write
  .mode("overwrite")
  .format("delta")
  .saveAsTable("workspace.portfolio_jobs.gold_top_companies"))


In [0]:
desc_col = "job_summary" if "job_summary" in s.columns else ("description" if "description" in s.columns else None)

if desc_col:
    s2 = s.withColumn(
        "is_remote",
        F.lower(F.col(desc_col)).rlike(r"\b(remote|work from home|wfh)\b").cast("boolean")
    )
    gold_remote = (s2.groupBy("is_remote").agg(F.count("*").alias("job_posts")))
    (gold_remote.write
      .mode("overwrite")
      .format("delta")
      .saveAsTable("workspace.portfolio_jobs.gold_remote_share"))